##### 4_featureEngineering

This notebook takes the csv that was compiled from the notebook 3_compileDischargesHospitalCensusData and does further processing on it such as: 
- Dealing with Categorical Variables

- Feature Engineering

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
aprHospFeat = pd.read_csv('dataFiles/compiledDischargesHospitalFeaturesCensus.csv', low_memory=False)

In [3]:
aprHospFeat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2209446 entries, 0 to 2209445
Data columns (total 40 columns):
Unnamed: 0                                              int64
Health Service Area                                     object
Facility Id                                             float64
Length of Stay                                          object
Type of Admission                                       object
APR DRG Code                                            int64
APR DRG Description                                     object
APR Severity of Illness Code                            float64
Payment Typology 1                                      object
Payment Typology 2                                      object
Payment Typology 3                                      object
Emergency Department Indicator                          object
Total Charges                                           float64
Total Costs                                             float64
yr 

#### Recoding the categorical vars into OneHot

##### Payment Types 1,2,3
There are 3 columns for payment types. This variable is looking at the difference payment types across the 3 columns and inputing a "1" for the various types.

In [4]:
aprHospFeat['Payment Typology 1'].unique()[:-1]

array(['Medicare', 'Blue Cross/Blue Shield', 'Private Health Insurance',
       'Self-Pay', 'Medicaid', 'Federal/State/Local/VA',
       'Miscellaneous/Other', 'Department of Corrections',
       'Managed Care, Unspecified'], dtype=object)

In [7]:
for i in range(len(aprHospFeat['Payment Typology 1'].unique())-1):
    aprHospFeat.loc[(aprHospFeat['Payment Typology 1'] == aprHospFeat['Payment Typology 1'].unique()[i]) | 
                (aprHospFeat['Payment Typology 2'] == aprHospFeat['Payment Typology 1'].unique()[i]) |
                (aprHospFeat['Payment Typology 3'] == aprHospFeat['Payment Typology 1'].unique()[i]), 
                    'payment_'+aprHospFeat['Payment Typology 1'].unique()[i]] = 1

In [8]:
 aprHospFeat.filter(like = 'payment').fillna(0, inplace=True)

/nfshome/yg833/.conda/envs/mgPUI/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


##### Type of Admission
One hot encoding the type of admission. 

In [6]:
aprHospFeat['Type of Admission'].unique()

array(['Urgent', 'Elective', 'Emergency', 'Newborn', 'Not Available',
       'Trauma'], dtype=object)

In [7]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['Type of Admission'])

##### Emergency Department Indicator

In [8]:
aprHospFeat['Emergency Department Indicator'].unique()

array(['Y', 'N'], dtype=object)

In [9]:
aprHospFeat.loc[aprHospFeat['Emergency Department Indicator'] == 'Y', 'emergencyRoom' ]= 1

In [10]:
aprHospFeat['emergencyRoom'].fillna(0, inplace=True)

##### Hospital Ownership

In [11]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['Hospital Ownership'])

##### Rural vs Urban Setting

In [12]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['rural_versus_urban'])

In [13]:
aprHospFeat.drop(['Payment Typology 1', 'Payment Typology 2', 'Payment Typology 3', 
                 'Emergency Department Indicator'], 
                axis=1, inplace=True)

#### Feature Engineering & Creating New Variables

##### 1. ratio # insured to # total population

In [14]:
aprHospFeat['ratioInsuredTotalPopulation'] = aprHospFeat['totalInsured']/ aprHospFeat['totalPopulation']

##### 2. ratio # fertile women to # total population

In [15]:
aprHospFeat['ratioFertilityTotalPopulation'] = aprHospFeat['totalFertile']/ aprHospFeat['totalPopulation']

##### 3. ratio total charges to total costs

In [16]:
aprHospFeat['ratioChargesCosts'] = aprHospFeat['Total Charges']/ aprHospFeat['Total Costs']

##### 4. charges per day of stay

In [17]:
aprHospFeat.loc[aprHospFeat['Length of Stay']== '120 +', 'days'] = 120

In [18]:
aprHospFeat.loc[aprHospFeat['Length of Stay'] != '120 +', 'days'] = aprHospFeat['Length of Stay']

In [19]:
aprHospFeat['days'] = pd.to_numeric(aprHospFeat['days'])

In [20]:
aprHospFeat['chargesPerDay'] = aprHospFeat['Total Charges']/ aprHospFeat['days']

In [21]:
aprHospFeat['Facility Id'].unique()

array([3.700e+01, 3.900e+01, 6.600e+01, 9.800e+01, 1.030e+02, 1.110e+02,
       1.140e+02,       nan, 1.280e+02, 1.650e+02, 1.740e+02, 1.350e+02,
       3.250e+02, 3.030e+02, 3.240e+02, 6.760e+02, 6.780e+02, 7.980e+02,
       8.040e+02, 8.120e+02, 8.150e+02, 2.070e+02, 2.100e+02, 2.130e+02,
       2.180e+02, 2.670e+02, 2.800e+02, 3.390e+02, 5.650e+02, 5.830e+02,
       1.180e+02, 1.160e+02, 8.660e+02, 5.740e+02, 1.153e+03, 3.930e+02,
       4.090e+02, 4.110e+02, 4.130e+02, 4.140e+02, 4.710e+02, 8.700e+02,
       8.580e+02, 8.730e+02, 1.028e+03, 4.200e+01, 4.300e+01, 8.500e+01,
       1.580e+02, 3.970e+02, 4.010e+02, 3.670e+02, 3.770e+02, 3.790e+02,
       3.830e+02, 3.620e+02, 5.890e+02, 5.980e+02, 5.990e+02, 6.300e+02,
       6.360e+02, 6.350e+02, 7.270e+02, 9.770e+02, 1.000e+00, 5.000e+00,
       4.000e+00, 1.460e+02, 3.300e+02, 4.840e+02, 7.560e+02, 7.550e+02,
       9.250e+03, 8.180e+02, 8.290e+02, 1.005e+03, 1.810e+02, 6.990e+02,
       7.080e+02, 6.940e+02, 7.520e+02, 7.760e+02, 

In [22]:
aprHospFeat.head()

,Unnamed: 0,Health Service Area,Facility Id,Length of Stay,APR DRG Code,APR DRG Description,APR Severity of Illness Code,Total Charges,Total Costs,yr,...,Hospital Ownership_Voluntary non-profit - Church,Hospital Ownership_Voluntary non-profit - Other,Hospital Ownership_Voluntary non-profit - Private,rural_versus_urban_R,rural_versus_urban_U,ratioInsuredTotalPopulation,ratioFertilityTotalPopulation,ratioChargesCosts,days,chargesPerDay
0,0,Western NY,37.0,8,139.0,Other pneumonia,3.0,5621.25,5037.17,2013.0,...,0,0,1,1,0,0.989499,0.218558,1.115954,8.0,702.656250
1,1,Western NY,37.0,3,139.0,Other pneumonia,2.0,2435.75,2271.92,2013.0,...,0,0,1,1,0,0.989499,0.218558,1.072111,3.0,811.916667
2,2,Western NY,37.0,4,139.0,Other pneumonia,2.0,3754.95,3579.69,2013.0,...,0,0,1,1,0,0.989499,0.218558,1.048960,4.0,938.737500
3,3,Western NY,37.0,2,139.0,Other pneumonia,1.0,2096.05,2195.02,2013.0,...,0,0,1,1,0,0.989499,0.218558,0.954912,2.0,1048.025000
4,4,Western NY,37.0,1,139.0,Other pneumonia,3.0,1583.25,1772.56,2013.0,...,0,0,1,1,0,0.989499,0.218558,0.893200,1.0,1583.250000


##### 5. average number of days per year per facility id per drg

In [28]:
aprHospFeat.merge(, 
                  left_on=['yr','Facility Id' ,'APR DRG Code'], right_on=['yr','Facility Id' ,'APR DRG Code'])

ValueError: No axis named Facility Id for object type <class 'pandas.core.frame.DataFrame'>

In [33]:
aprHospFeat.groupby('yr','Provider ID','APR DRG Code').mean()['days']

ValueError: No axis named Provider ID for object type <class 'pandas.core.frame.DataFrame'>

##### 6. Number of Payments Used

In [26]:
aprHospFeat['numberPaymentTypes'] = aprHospFeat.filter(like = 'payment').sum(axis=1)

##### 7. Number of discharges per year per facility id by APR DRG that is in Urgent, Emergency and Trauma

In [ ]:
aprHospFeat[(aprHospFeat['Type of Admission_Emergency'] == 1) |
           (aprHospFeat['Type of Admission_Trauma'] == 1) |
           (aprHospFeat['Type of Admission_Urgent'] == 1)].groupby('yr', )

In [25]:
aprHospFeat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551443 entries, 0 to 2551442
Data columns (total 66 columns):
Unnamed: 0                                                        int64
Health Service Area                                               object
Facility Id                                                       float64
Length of Stay                                                    object
APR DRG Code                                                      float64
APR DRG Description                                               object
APR Severity of Illness Code                                      float64
Total Charges                                                     float64
Total Costs                                                       float64
yr                                                                float64
Provider ID                                                       float64
Hospital overall rating                                           float64
Mort